In [18]:
import csv
import numpy as np

def csv_to_numeric_array(filename):
    data = []
    with open(filename, newline="") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            converted_row = []
            for item in row:
                try:
                    # Try converting to float (covers both int & decimal)
                    converted_row.append(float(item))
                except ValueError:
                    # Skip non-numeric elements
                    continue
            if converted_row:   # only add non-empty rows
                data.append(converted_row)
    return data

# Load data from CSV files
arr1 = csv_to_numeric_array("W.csv")
arr2 = csv_to_numeric_array("b.csv")

print("W:", arr1)
print("b:", arr2)

def initialize_parameters(arr1, arr2):
    # Flatten the arrays
    arr1_flat = [item for sublist in arr1 for item in sublist]
    arr2_flat = [item for sublist in arr2 for item in sublist]
    
    # Reshape weights according to network architecture
    W1 = np.array(arr1_flat[0:6*64]).reshape(64, 6)
    W2 = np.array(arr1_flat[6*64:6*64+64*32]).reshape(32, 64)
    W3 = np.array(arr1_flat[6*64+64*32:6*64+64*32+32*16]).reshape(16, 32)
    W4 = np.array(arr1_flat[6*64+64*32+32*16:6*64+64*32+32*16+16*1]).reshape(1, 16)
    
    # Extract biases
    B1 = np.array(arr2_flat[0:64]).reshape(64, 1)
    B2 = np.array(arr2_flat[64:64+32]).reshape(32, 1)
    B3 = np.array(arr2_flat[64+32:64+32+16]).reshape(16, 1)
    B4 = np.array(arr2_flat[64+32+16:64+32+16+1]).reshape(1, 1)
    
    return W1, B1, W2, B2, W3, B3, W4, B4

def ReLU(Z):
    return np.maximum(0, Z)

def sigmoid(Z):
    # Clip Z to avoid overflow in exp
    Z = np.clip(Z, -500, 500)
    return 1 / (1 + np.exp(-Z))

def forward_propagation(X, W1, B1, W2, B2, W3, B3, W4, B4):
    # Reshape X to be a column vector
    X = X.reshape(-1, 1)
    
    Z1 = np.dot(W1, X) + B1
    A1 = ReLU(Z1)
    
    Z2 = np.dot(W2, A1) + B2
    A2 = ReLU(Z2)
    
    Z3 = np.dot(W3, A2) + B3
    A3 = ReLU(Z3)
    
    Z4 = np.dot(W4, A3) + B4
    A4 = sigmoid(Z4)
    
    return Z1, A1, Z2, A2, Z3, A3, Z4, A4

def compute_cost(A4, Y):
    m = Y.shape[1]
    # Clip A4 to avoid log(0)
    A4 = np.clip(A4, 1e-10, 1 - 1e-10)
    cost = -np.sum(Y * np.log(A4) + (1 - Y) * np.log(1 - A4)) / m
    return cost

def backward_propagation(X, Y, W1, B1, W2, B2, W3, B3, W4, B4, Z1, A1, Z2, A2, Z3, A3, Z4, A4):
    m = 1  # Since we're processing one sample at a time
    
    # Reshape X and Y
    X = X.reshape(-1, 1)
    Y = Y.reshape(1, 1)
    
    # Output layer gradient
    dZ4 = A4 - Y
    dW4 = (1/m) * np.dot(dZ4, A3.T)
    dB4 = (1/m) * np.sum(dZ4, axis=1, keepdims=True)
    
    # Third hidden layer gradient
    dA3 = np.dot(W4.T, dZ4)
    dZ3 = dA3 * (Z3 > 0)
    dW3 = (1/m) * np.dot(dZ3, A2.T)
    dB3 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)
    
    # Second hidden layer gradient
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = dA2 * (Z2 > 0)
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    dB2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    
    # First hidden layer gradient
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * (Z1 > 0)
    dW1 = (1/m) * np.dot(dZ1, X.T)
    dB1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, dB1, dW2, dB2, dW3, dB3, dW4, dB4

def update_parameters(W1, B1, W2, B2, W3, B3, W4, B4, dW1, dB1, dW2, dB2, dW3, dB3, dW4, dB4, learning_rate):
    W1 -= learning_rate * dW1
    B1 -= learning_rate * dB1
    W2 -= learning_rate * dW2
    B2 -= learning_rate * dB2
    W3 -= learning_rate * dW3
    B3 -= learning_rate * dB3
    W4 -= learning_rate * dW4
    B4 -= learning_rate * dB4
    
    return W1, B1, W2, B2, W3, B3, W4, B4

def get_predictions(A4):
    return (A4 > 0.5).astype(int)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)

def gradient_descent(X, Y, alpha, iterations, arr1, arr2):
    W1, B1, W2, B2, W3, B3, W4, B4 = initialize_parameters(arr1, arr2)
    
    for i in range(iterations):
        # Forward propagation
        Z1, A1, Z2, A2, Z3, A3, Z4, A4 = forward_propagation(X, W1, B1, W2, B2, W3, B3, W4, B4)
        
        # Compute cost
        cost = compute_cost(A4, Y.reshape(1, -1))
        
        # Backward propagation
        dW1, dB1, dW2, dB2, dW3, dB3, dW4, dB4 = backward_propagation(
            X, Y, W1, B1, W2, B2, W3, B3, W4, B4, Z1, A1, Z2, A2, Z3, A3, Z4, A4
        )
        
        # Update parameters
        W1, B1, W2, B2, W3, B3, W4, B4 = update_parameters(
            W1, B1, W2, B2, W3, B3, W4, B4, 
            dW1, dB1, dW2, dB2, dW3, dB3, dW4, dB4, alpha
        )
        
        if i % 100 == 0:
            predictions = get_predictions(A4)
            accuracy = get_accuracy(predictions, Y.reshape(1, -1))
            print(f"Iteration {i}, Cost: {cost:.4f}, Accuracy: {accuracy:.4f}")
    
    return W1, B1, W2, B2, W3, B3, W4, B4

# Training data
x_train = np.array([1/9, 2/9, 3/9, 4/9, 5/9, 6/9])
y_train = np.array([0])

# Train the network
W1, B1, W2, B2, W3, B3, W4, B4 = gradient_descent(x_train, y_train, 0.1, 1000, arr1, arr2)

W: [[0.471435159, -1.190975666, 1.432706952, -0.312651902, -0.720588744, 0.887162924, 0.859588385, -0.636523485, 0.015696373, -2.242684841, 1.150035739, 0.991946042, 0.953324139, -2.021254778, -0.334077358, 0.002118365, 0.405453414, 0.289091945, 1.321158171, -1.546905518, -0.20264633, -0.655969322, 0.193421379, 0.553438902, 1.318151593, -0.469305277, 0.675554097, -1.817027211, -0.183108538, 1.05896914, -0.397840232, 0.33743766, 1.047578573, 1.045938253, 0.863717318, -0.122091576, 0.124712951, -0.322794795, 0.841674685, 2.390960455, 0.076199591, -0.566445947, 0.036141936, -2.074977636, 0.247792199, -0.897156775, -0.136794835, 0.018289192, 0.755414009, 0.215268582, 0.841008782, -1.44581008, -1.401973248, -0.100918204, -0.54824245, -0.14461951, 0.354020327, -0.035513025, 0.56573832, 1.545658827, -0.97423631, -0.07034488, 0.307968855, -0.208498761], [0.291205347, 0.566533685, 0.503591776, 0.285295695, 0.484288126, 1.363481522, -0.78110528, -0.468017668, 1.224574327, -1.28110826, 0.87547552

In [19]:
# Function to save weights and biases in the required CSV format
def save_updated_parameters(W1, B1, W2, B2, W3, B3, W4, B4):
    with open('dw_123456.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # W1
        matrix = W1.T  # Transpose back to (6,64)
        for row in matrix:
            writer.writerow(['{:.16f}'.format(val) for val in row])
        # W2
        matrix = W2.T  # (64,32)
        for row in matrix:
            writer.writerow(['{:.16f}'.format(val) for val in row])
        # W3
        matrix = W3.T  # (32,16)
        for row in matrix:
            writer.writerow(['{:.16f}'.format(val) for val in row])
        # W4
        matrix = W4.T  # (16,1)
        for row in matrix:
            writer.writerow(['{:.16f}'.format(val) for val in row])
    
    with open('db_123456.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['{:.16f}'.format(val) for val in B1.flatten()])
        writer.writerow(['{:.16f}'.format(val) for val in B2.flatten()])
        writer.writerow(['{:.16f}'.format(val) for val in B3.flatten()])
        writer.writerow(['{:.16f}'.format(val) for val in B4.flatten()])

# Call the save function
save_updated_parameters(W1, B1, W2, B2, W3, B3, W4, B4)

In [6]:
import numpy as np
import csv

class NeuralNetwork:
    def __init__(self, weights_file, biases_file):
        # Load weights and biases from CSV files
        self.weights = self.load_weights(weights_file)
        self.biases = self.load_biases(biases_file)
        
        # Define activation functions
        self.relu = lambda x: np.maximum(0, x)
        self.sigmoid = lambda x: 1 / (1 + np.exp(-x))
        
        # Define derivatives of activation functions
        self.relu_derivative = lambda x: (x > 0).astype(np.float32)
        self.sigmoid_derivative = lambda x: x * (1 - x)
    
    def load_weights(self, filename):
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            data = [list(map(np.float32, row)) for row in reader]
        
        # Reshape weights according to network architecture
        # Input to Hidden1: 6x64
        w1 = np.array(data[0:6])
        # Hidden1 to Hidden2: 64x32
        w2 = np.array(data[6:6+64])
        # Hidden2 to Hidden3: 32x16
        w3 = np.array(data[6+64:6+64+32])
        # Hidden3 to Output: 16x1
        w4 = np.array(data[6+64+32:6+64+32+16])
        
        return [w1, w2, w3, w4]
    
    def load_biases(self, filename):
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            data = [list(map(np.float32, row)) for row in reader]
        
        # Reshape biases according to network architecture
        # Hidden1: 64
        b1 = np.array(data[0]).reshape(-1, 1)
        # Hidden2: 32
        b2 = np.array(data[1]).reshape(-1, 1)
        # Hidden3: 16
        b3 = np.array(data[2]).reshape(-1, 1)
        # Output: 1
        b4 = np.array(data[3]).reshape(-1, 1)
        
        return [b1, b2, b3, b4]
    
    def forward_pass(self, x):
        # Store intermediate values for backpropagation
        self.activations = [x.reshape(-1, 1)]
        self.z_values = []
        
        # Hidden Layer 1
        z1 = self.weights[0].T @ self.activations[0] + self.biases[0]
        a1 = self.relu(z1)
        self.z_values.append(z1)
        self.activations.append(a1)
        
        # Hidden Layer 2
        z2 = self.weights[1].T @ a1 + self.biases[1]
        a2 = self.relu(z2)
        self.z_values.append(z2)
        self.activations.append(a2)
        
        # Hidden Layer 3
        z3 = self.weights[2].T @ a2 + self.biases[2]
        a3 = self.relu(z3)
        self.z_values.append(z3)
        self.activations.append(a3)
        
        # Output Layer
        z4 = self.weights[3].T @ a3 + self.biases[3]
        a4 = self.sigmoid(z4)
        self.z_values.append(z4)
        self.activations.append(a4)
        
        return a4
    
    def backpropagation(self):
        # Initialize gradients
        dw = [np.zeros_like(w) for w in self.weights]
        db = [np.zeros_like(b) for b in self.biases]
        
        # Output layer gradient
        delta = 1  # ∂output/∂output = 1
        delta *= self.sigmoid_derivative(self.activations[4])  # ∂output/∂z4
        
        db[3] = delta
        dw[3] = self.activations[3] @ delta.T
        
        # Hidden Layer 3
        delta = self.weights[3] @ delta  # ∂output/∂a3
        delta *= self.relu_derivative(self.z_values[2])  # ∂output/∂z3
        
        db[2] = delta
        dw[2] = self.activations[2] @ delta.T
        
        # Hidden Layer 2
        delta = self.weights[2] @ delta  # ∂output/∂a2
        delta *= self.relu_derivative(self.z_values[1])  # ∂output/∂z2
        
        db[1] = delta
        dw[1] = self.activations[1] @ delta.T
        
        # Hidden Layer 1
        delta = self.weights[1] @ delta  # ∂output/∂a1
        delta *= self.relu_derivative(self.z_values[0])  # ∂output/∂z1
        
        db[0] = delta
        dw[0] = self.activations[0] @ delta.T
        
        return dw, db
    
    def preprocess_input(self, index_number):
        # Split into digits and normalize
        digits = [int(d) for d in str(index_number)]
        return np.array(digits, dtype=np.float32) / 9.0
    
    def compute_gradients(self, index_number):
        # Preprocess input
        x = self.preprocess_input(index_number)
        
        # Forward pass
        self.forward_pass(x)
        
        # Backpropagation
        dw, db = self.backpropagation()
        
        return dw, db

def save_gradients(dw, db, dw_filename, db_filename):
    # Save weight gradients
    with open(dw_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for layer in dw:
            for row in layer:
                writer.writerow([f"{x:.16f}" for x in row])
    
    # Save bias gradients
    with open(db_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for layer in db:
            writer.writerow([f"{x:.16f}" for x in layer.flatten()])

# Example usage
if __name__ == "__main__":
    # Replace with your index number
    index_number = 123456
    
    # Initialize neural network
    nn = NeuralNetwork('W.csv', 'b.csv')
    
    # Compute gradients
    dw, db = nn.compute_gradients(index_number)
    
    # Save gradients to CSV files
    save_gradients(dw, db, 'dw.csv', 'db.csv')
    
    print("Gradients computed and saved successfully!")

ValueError: could not convert string to float: 'weights btw layer0 to layer1'

In [7]:
import numpy as np
import pandas as pd

df=pd.read_csv('W.csv')
df.head()



,weights btw layer0 to layer1,0.471435159,-1.190975666,1.432706952,-0.312651902,-0.720588744,0.887162924,0.859588385,-0.636523485,0.015696373,...,-0.54824245,-0.14461951,0.354020327,-0.035513025,0.56573832,1.545658827,-0.97423631,-0.07034488,0.307968855,-0.208498761
0,weights btw layer0 to layer1,0.291205,0.566534,0.503592,0.285296,0.484288,1.363482,-0.781105,-0.468018,1.224574,...,1.357797,-0.804834,-2.123620,-0.333502,-0.886719,0.334198,0.536784,-0.743830,-0.320204,-0.916199
1,weights btw layer0 to layer1,-0.319561,-0.619993,0.156998,-0.571455,1.057633,-0.791489,-0.524627,0.071878,1.910759,...,0.003760,-1.268994,-0.838843,0.553921,-0.504043,-0.788440,1.529401,0.205455,0.313013,0.866521
2,weights btw layer0 to layer1,0.527046,0.711112,-0.217545,2.637791,-1.742138,-0.094435,1.431184,0.592758,0.170297,...,-1.337968,-0.041387,0.821048,2.097801,1.282933,0.270338,1.003140,1.078674,0.340753,-0.198075
3,weights btw layer0 to layer1,-0.226632,-0.923831,0.355839,-1.270064,-0.195472,-0.463419,0.989415,1.388647,1.087714,...,-0.588227,0.025942,-0.572302,1.105587,-1.208016,-1.339701,2.075185,-1.489410,-1.150953,0.419060
4,weights btw layer0 to layer1,0.523546,0.821517,-0.148742,-0.045370,-0.704493,1.730316,0.228927,1.092741,-0.353572,...,-0.050050,0.743953,-1.724050,0.089902,0.322740,-0.377149,-0.000221,1.050114,-0.168699,0.797876


In [8]:
df1=pd.read_csv('b.csv')
df1.head()

,bias for layer1,-0.635154963,-0.681020141,-1.501585722,-0.368925124,-0.352999359,0.127384394,0.517531335,0.747836828,-0.809377551,...,-0.749805033,-0.267733008,0.631269991,-0.087142929,-0.069249019,-0.167822808,0.649498701,0.620435476,0.144408464,1.160045505
0,bias for layer2,0.665451,1.426556,1.158077,-0.496191,-0.010350,0.617206,0.336297,-0.358238,-0.995016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bias for layer3,-1.027926,-1.870784,0.661100,-1.804633,1.165536,0.663858,0.156078,0.413614,1.211116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bias for layer4,1.508889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import numpy as np
import csv

class NeuralNetwork:
    def __init__(self, weights_file, biases_file):
        # Load weights and biases from CSV files
        self.weights = self.load_weights(weights_file)
        self.biases = self.load_biases(biases_file)
        
        # Define activation functions
        self.relu = lambda x: np.maximum(0, x)
        self.sigmoid = lambda x: 1 / (1 + np.exp(-x))
        
        # Define derivatives of activation functions
        self.relu_derivative = lambda x: (x > 0).astype(np.float32)
        self.sigmoid_derivative = lambda x: x * (1 - x)
    
    def load_weights(self, filename):
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            data = []
            for row in reader:
                # Skip the first column (index column)
                if row:  # Check if row is not empty
                    data.append(list(map(np.float32, row[1:])))
        
        # Reshape weights according to network architecture
        # Input to Hidden1: 6x64
        w1 = np.array(data[0:6])
        # Hidden1 to Hidden2: 64x32
        w2 = np.array(data[6:6+64])
        # Hidden2 to Hidden3: 32x16
        w3 = np.array(data[6+64:6+64+32])
        # Hidden3 to Output: 16x1
        w4 = np.array(data[6+64+32:6+64+32+16])
        
        return [w1, w2, w3, w4]
    
    def load_biases(self, filename):
        with open(filename, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            data = []
            for row in reader:
                # Skip the first column (index column)
                if row:  # Check if row is not empty
                    data.append(list(map(np.float32, row[1:])))
        
        # Reshape biases according to network architecture
        # Hidden1: 64
        b1 = np.array(data[0]).reshape(-1, 1)
        # Hidden2: 32
        b2 = np.array(data[1]).reshape(-1, 1)
        # Hidden3: 16
        b3 = np.array(data[2]).reshape(-1, 1)
        # Output: 1
        b4 = np.array(data[3]).reshape(-1, 1)
        
        return [b1, b2, b3, b4]
    
    def forward_pass(self, x):
        # Store intermediate values for backpropagation
        self.activations = [x.reshape(-1, 1)]
        self.z_values = []
        
        # Hidden Layer 1
        z1 = self.weights[0].T @ self.activations[0] + self.biases[0]
        a1 = self.relu(z1)
        self.z_values.append(z1)
        self.activations.append(a1)
        
        # Hidden Layer 2
        z2 = self.weights[1].T @ a1 + self.biases[1]
        a2 = self.relu(z2)
        self.z_values.append(z2)
        self.activations.append(a2)
        
        # Hidden Layer 3
        z3 = self.weights[2].T @ a2 + self.biases[2]
        a3 = self.relu(z3)
        self.z_values.append(z3)
        self.activations.append(a3)
        
        # Output Layer
        z4 = self.weights[3].T @ a3 + self.biases[3]
        a4 = self.sigmoid(z4)
        self.z_values.append(z4)
        self.activations.append(a4)
        
        return a4
    
    def backpropagation(self):
        # Initialize gradients
        dw = [np.zeros_like(w) for w in self.weights]
        db = [np.zeros_like(b) for b in self.biases]
        
        # Output layer gradient
        delta = 1  # ∂output/∂output = 1
        delta *= self.sigmoid_derivative(self.activations[4])  # ∂output/∂z4
        
        db[3] = delta
        dw[3] = self.activations[3] @ delta.T
        
        # Hidden Layer 3
        delta = self.weights[3] @ delta  # ∂output/∂a3
        delta *= self.relu_derivative(self.z_values[2])  # ∂output/∂z3
        
        db[2] = delta
        dw[2] = self.activations[2] @ delta.T
        
        # Hidden Layer 2
        delta = self.weights[2] @ delta  # ∂output/∂a2
        delta *= self.relu_derivative(self.z_values[1])  # ∂output/∂z2
        
        db[1] = delta
        dw[1] = self.activations[1] @ delta.T
        
        # Hidden Layer 1
        delta = self.weights[1] @ delta  # ∂output/∂a1
        delta *= self.relu_derivative(self.z_values[0])  # ∂output/∂z1
        
        db[0] = delta
        dw[0] = self.activations[0] @ delta.T
        
        return dw, db
    
    def preprocess_input(self, index_number):
        # Split into digits and normalize
        digits = [int(d) for d in str(index_number)]
        return np.array(digits, dtype=np.float32) / 9.0
    
    def compute_gradients(self, index_number):
        # Preprocess input
        x = self.preprocess_input(index_number)
        
        # Forward pass
        self.forward_pass(x)
        
        # Backpropagation
        dw, db = self.backpropagation()
        
        return dw, db

def save_gradients(dw, db, dw_filename, db_filename):
    # Save weight gradients
    with open(dw_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for layer in dw:
            for row in layer:
                writer.writerow([f"{x:.16f}" for x in row])
    
    # Save bias gradients
    with open(db_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for layer in db:
            writer.writerow([f"{x:.16f}" for x in layer.flatten()])

# Example usage
if __name__ == "__main__":
    # Replace with your index number
    index_number = 123456
    
    # Initialize neural network
    nn = NeuralNetwork('W.csv', 'b.csv')
    
    # Compute gradients
    dw, db = nn.compute_gradients(index_number)
    
    # Save gradients to CSV files
    save_gradients(dw, db, 'dw.csv', 'db.csv')
    
    print("Gradients computed and saved successfully!")

ValueError: could not convert string to float: ''